In [1]:
import fitsio
from astropy.io import fits
from astropy.table import Table, vstack, join
import numpy as np
import h5py
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [2]:
def calc_gold_cuts(cat, flags_gold_colname='FLAGS_GOLD_SOF_ONLY'):
    gold_cuts = np.where( (cat['flags_foreground'] == 0) &
                          (cat['flags_badregions'] < 2) &
                          (cat['flags_footprint'] == 1) &
                          (cat[flags_gold_colname] < 2)
                        )
    return gold_cuts

def calc_gold_photometry_cuts(cat, flags_gold_colname='FLAGS_GOLD_SOF_ONLY'):
    gold_phot_cuts = np.where( (cat['flags_foreground'] == 0) &
                          (cat['flags_badregions'] < 2) &
                          (cat['flags_footprint'] == 1) &
                          (cat[flags_gold_colname] < 8)
                        )
    return gold_phot_cuts

def calc_shape_cuts(cat):
    shape_cuts = np.where( (cat['flags'] == 0) &
                           (cat['size_ratio'] > 0.5) &
                           (cat['snr'] > 10) &
                           (cat['snr'] < 1000)
                          )
    return shape_cuts

def apply_sompz_cuts(self, use_match_flag=True):
    '''
    This is the same as "selection2"
    '''
    self._check_for_cols(self._sompz_cut_cols)

    # Mag & color cuts
    color_cuts = np.where( (cat['mag_i'] >= 18.) &
                           (cat['mag_i'] <= 23.5) &
                           (cat['mag_r'] >= 15.) &
                           (cat['mag_r'] <= 26.) &
                           (cat['mag_z'] >= 15.) &
                           (cat['mag_z'] <= 26.) &
                           ((cat['mag_z'] - cat['mag_i']) <= 1.5) &
                           ((cat['mag_z'] - cat['mag_i']) >= -4.) &
                           ((cat['mag_r'] - cat['mag_i']) <= 4.) &
                           ((cat['mag_r'] - cat['mag_i']) >= -1.5)
                          )

    self.apply_cut(color_cuts)

    # Binary star cut, taken from Alex A.
    highe_cut = np.greater(np.sqrt(np.power(cat['e_1'],2.)
                           + np.power(cat['e_2'],2)), 0.8)

    c = 22.5
    m = 3.5

    magT_cut = np.log10(cat['T']) < (c - self.flux2mag(cat['flux_r'])) / m

    binaries = highe_cut * magT_cut

    return ~binaries
    
    return 

def calc_match_flag_cut(cat, match_flag_col='match_flag_1.5_asec'):
    match_flag_cut = np.where(cat[match_flag_col] < 2)
    return match_flag_cut


In [3]:
run = 'y3-merged'
ver = '1.2'

In [4]:
match_file = '/project/projectdirs/des/severett/Balrog/{}/stacked_catalogs/{}/sof/balrog_matched_catalog_sof_{}_v{}.fits'.format(run, ver, run, ver)
det_file = '/project/projectdirs/des/severett/Balrog/{}/stacked_catalogs/{}/sof/balrog_detection_catalog_sof_{}_v{}.fits'.format(run, ver, run, ver)
mcal_file = '/project/projectdirs/des/severett/Balrog/{}/stacked_catalogs/{}/mcal/balrog_mcal_stack-y3v02-0-riz-noNB-mcal_{}_v{}.h5'.format(run, ver, run, ver)

with fits.open(match_file) as hdu:
    match_cols = hdu[1].columns.names
    
with fits.open(det_file) as hdu:
    det_cols = hdu[1].columns.names
    
# with h5py.File(mcal_file) as mcal:
    

In [13]:
print('Loading det...')
det = pd.DataFrame(fitsio.read(det_file, columns=['bal_id']).byteswap().newbyteorder())
det['det_index'] = det.index
print(f'len(det) = {len(det)}')
print(det)

Loading det...
len(det) = 26442133
                    bal_id  det_index
0             100156035400          0
1             102037044570          1
2             100530030400          2
3             100010040400          3
4             100521042490          4
...                    ...        ...
26442128  1005410224913037   26442128
26442129  1002380441413081   26442129
26442130  1003480491413068   26442130
26442131  1003060324913071   26442131
26442132  1006000391413076   26442132

[26442133 rows x 2 columns]


In [14]:
print('Loading match...')
match = pd.DataFrame(fitsio.read(match_file, columns=['bal_id']).byteswap().newbyteorder())
match['match_index'] = match.index
print(f'len(match) = {len(match)}')
print(match)

Loading match...
len(match) = 11250845
                    bal_id  match_index
0          100402021246920            0
1          100402021248530            1
2          100402021246434            2
3          100402021246217            3
4          100402021245800            4
...                    ...          ...
11250840  1000400233212826     11250840
11250841   100040023321450     11250841
11250842   100040023325968     11250842
11250843   100040023329128     11250843
11250844   100040023326952     11250844

[11250845 rows x 2 columns]


In [16]:
print('joining...')
joined = match.merge(det, how='inner', on='bal_id')
print(f'len(joined) = {len(joined)}')
print(joined)

joining...
len(joined) = 11250845
                    bal_id  match_index  det_index
0          100402021246920            0   13941048
1          100402021248530            1   17110873
2          100402021246434            2   12989101
3          100402021246217            3   12539828
4          100402021245800            4   11658202
...                    ...          ...        ...
11250840  1000400233212826     11250840   25926536
11250841   100040023321450     11250841    3066081
11250842   100040023325968     11250842   12325810
11250843   100040023329128     11250843   18737331
11250844   100040023326952     11250844   14232807

[11250845 rows x 3 columns]


In [ ]:
joined.write()

In [9]:
with h5py.File(mcal_file) as mcal:
    print(mcal['catalog'].keys())

<KeysViewHDF5 ['sheared_1m', 'sheared_1p', 'sheared_2m', 'sheared_2p', 'unsheared']>


In [ ]:
mastercat_file = '/project/projectdirs/des/severett/Balrog/{}/stacked_catalogs/{}/sof/balrog_mastercat_sof_{}_v{}.fits'.format(run, ver, run, ver)

if os.path.exists(mastercat_file):
    os.remove(mastercat_file)

with h5py.File(mastercat_file, 'rw') as mastercat:
    mastercat.create_group('catalog')
    mastercat.create_group('index')
    
        
    

In [5]:
data_file = '/project/projectdirs/des/www/y3_cats/Y3_mastercat_03_31_20.h5'
with h5py.File(data_file, 'r') as mcat:
    print(mcat.keys())

<KeysViewHDF5 ['catalog', 'index', 'maps', 'masks', 'randoms', 'regions']>


In [10]:
!ls -sh '/project/projectdirs/des/severett/Balrog/y3-merged/stacked_catalogs/1.2/sof/'

total 40G
5.7G balrog_detection_catalog_sof_y3-merged_v1.2.fits  1.0K nz-comparison
 35G balrog_matched_catalog_sof_y3-merged_v1.2.fits


In [ ]:
!ls '/project/projectdirs/des/severett/Balrog/y3-merged/stacked_catalogs/1.2/sof/'